Imports

In [41]:
import pandas as pd
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

import os

Make sure we are in the right directory

In [42]:
# os.chdir('Documents/Deep-Learning-Chronicles/air_quality_prediction/')
# os.getcwd()

Load the data

In [43]:
df = pd.read_csv("smoke_occurence_nairobi.csv")
df.head()

,id,time_opening_windows,time_closing_windows,smoke_detected,time_sensing_smoke,duration,date,day,occassion,weather,type_of_smoke
0,1,1641,1717,NaN,1717.0,NaN,2025-10-03,Friday,nothing,cloudywithoutwind,stove
1,2,1252,1528,NaN,1728.0,NaN,2025-10-04,Saturday,NaN,cloudywithoutwind,stove
2,3,1452,1734,NaN,1734.0,NaN,2025-10-05,Sunday,NaN,windy,NaN
3,4,1035,1759,NaN,1759.0,NaN,2025-10-06,NaN,NaN,windy,stove
4,5,1400,1813,True,1813.0,NaN,2025-10-07,Tuesday,NaN,NaN,NaN


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    20 non-null     int64  
 1   time_opening_windows  20 non-null     int64  
 2   time_closing_windows  20 non-null     int64  
 3   smoke_detected        14 non-null     object 
 4   time_sensing_smoke    8 non-null      float64
 5   duration              12 non-null     float64
 6   date                  20 non-null     object 
 7   day                   19 non-null     object 
 8   occassion             12 non-null     object 
 9   weather               16 non-null     object 
 10  type_of_smoke         6 non-null      object 
dtypes: float64(2), int64(3), object(6)
memory usage: 1.8+ KB


In [51]:
df.describe()

,id,time_opening_windows,time_closing_windows,smoke_detected,time_sensing_smoke,duration,window_start_min,window_end_min,window_length_min
count,20.00000,20.000000,20.000000,20.000000,8.000000,12.000000,20.000000,20.00000,20.000000
mean,10.50000,1213.750000,1625.500000,0.100000,1652.875000,0.833333,739.750000,985.50000,245.750000
std,5.91608,175.860553,137.070669,0.307794,196.247975,2.886751,106.304367,82.64668,114.360867
min,1.00000,934.000000,1224.000000,0.000000,1224.000000,0.000000,574.000000,744.00000,25.000000
25%,5.75000,1089.750000,1527.000000,0.000000,1662.750000,0.000000,659.750000,927.00000,180.000000
50%,10.50000,1179.500000,1625.500000,0.000000,1731.000000,0.000000,719.500000,985.50000,263.000000
75%,15.25000,1318.250000,1730.250000,0.000000,1750.750000,0.000000,798.250000,1050.25000,313.000000
max,20.00000,1641.000000,1813.000000,1.000000,1813.000000,10.000000,1001.000000,1093.00000,444.000000


Basic cleaning

In [45]:
df["smoke_detected"] = (
    df["smoke_detected"]
    .fillna(False)
    .astype(str)
    .str.lower()
    .map({"true": 1, "false": 0})
)

df["smoke_detected"].value_counts()

C:\Users\ianwr\AppData\Local\Temp\ipykernel_7396\3610187339.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(False)


smoke_detected
0    18
1     2
Name: count, dtype: int64

In [46]:
def time_to_minutes(t):
    if pd.isna(t) or t == "":
        return np.nan
    t = str(t).zfill(4)
    final = int(t[:2]) * 60 + int(t[2:])
    return final

df["window_start_min"] = df["time_opening_windows"].apply(time_to_minutes)
df["window_end_min"] = df["time_closing_windows"].apply(time_to_minutes)
df["window_length_min"] = df["window_end_min"] - df["window_start_min"]

df[["window_start_min", "window_end_min", "window_length_min"]].head()


,window_start_min,window_end_min,window_length_min
0,1001,1037,36
1,772,928,156
2,892,1054,162
3,635,1079,444
4,840,1093,253


Clean the categorical columns

In [47]:
categorical_cols = ["day", "weather", "occassion"]

for col in categorical_cols:
    df[col] = df[col].fillna("unknown").str.lower()


In [50]:
df[['smoke_detected']].describe()

,smoke_detected
count,20.000000
mean,0.100000
std,0.307794
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


#### Feature Selection
For now we do not need these fields: **id, date, time_sensing_smoke, duration, type_of_smoke**

According to my research, the above fields are too descriptive for a classification problem (version 1).

But we could surely use these fields as inputs:
- window_start_min
- window_end_min
- window_length_min
- day
- weather
- occassion

Target would be:
- smoke_detected

In [52]:
# Training pipeline

feature_cols = [
    "window_start_min",
    "window_end_min",
    "window_length_min",
    "day",
    "weather",
    "occassion"
]

X = df[feature_cols]
y = df["smoke_detected"]


In [53]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=41,
    stratify=y
)


In [54]:
# preprocessing and model pipeline 

numeric_features = [
    "window_start_min",
    "window_end_min",
    "window_length_min"
]

categorical_features = [
    "day",
    "weather",
    "occassion"
]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

model = LogisticRegression(
    class_weight="balanced",
    max_iter=1000
)

pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", model)
])


In [57]:
# train the model
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['window_start_min',
                                                   'window_end_min',
                                                   'window_length_min']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['day', 'weather',
                                                   'occassion'])])),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [58]:
# evaluate the model
y_pred = pipeline.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[3 2]
 [1 0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.60      0.67         5
           1       0.00      0.00      0.00         1

    accuracy                           0.50         6
   macro avg       0.38      0.30      0.33         6
weighted avg       0.62      0.50      0.56         6



Findings:  
- with the current data, the model cannot confidently generalize smoke patterns yet.
(this is due to scarcity of the data, but we can fix that with time)

- Initial model performance is constrained by limited positive samples, motivating continued data collection and iterative retraining.

### What can we do next?

We can try switching from ```pipeline.predict()``` to ```pipeline.predict_proba()```

Because the former forces a **binary decision** while the latter allows:
- early warning thresholds 
- risk tuning 
- fewer missed events later 


In [69]:
X_test

,window_start_min,window_end_min,window_length_min,day,weather,occassion
13,668,957,289,monday,rainy,unknown
18,619,990,371,saturday,sunny,holiday
0,1001,1037,36,friday,cloudywithoutwind,nothing
2,892,1054,162,sunday,windy,unknown
3,635,1079,444,unknown,windy,unknown
4,840,1093,253,tuesday,unknown,unknown


In [60]:
y_proba = pipeline.predict_proba(X_test)[:, 1]

for prob, actual in zip(y_proba, y_test):
    print(f"Predicted smoke probability: {prob:.2f} | Actual: {actual}")


Predicted smoke probability: 0.00 | Actual: 0
Predicted smoke probability: 0.00 | Actual: 0
Predicted smoke probability: 1.00 | Actual: 0
Predicted smoke probability: 1.00 | Actual: 0
Predicted smoke probability: 0.00 | Actual: 0
Predicted smoke probability: 0.00 | Actual: 1


In [61]:
threshold = 0.25
y_pred_custom = (y_proba >= threshold).astype(int)

print(confusion_matrix(y_test, y_pred_custom))
print(classification_report(y_test, y_pred_custom))


[[3 2]
 [1 0]]
              precision    recall  f1-score   support

           0       0.75      0.60      0.67         5
           1       0.00      0.00      0.00         1

    accuracy                           0.50         6
   macro avg       0.38      0.30      0.33         6
weighted avg       0.62      0.50      0.56         6



### **Conclusion**

The system is designed to prefer false alarms, but current model confidence is limited by sparse positive data (limited positive samples).

As additional data is collected, the model’s confidence and recall are expected to improve.

## **Turning this to a deployable artifact**

For now anything the web app needs must live inside ONE pipeline object. 

Good thing, that's the case ```Pipeline(preprocess -> model)``` 

First, we save and reload the trained pipeline 

In [62]:
import joblib

joblib.dump(pipeline, "smoke_prediction_pipeline_v1.joblib")


['smoke_prediction_pipeline_v1.joblib']

In [63]:
loaded_pipeline = joblib.load("smoke_prediction_pipeline_v1.joblib")

# quick test
sample = X.iloc[[0]]
loaded_pipeline.predict(sample), loaded_pipeline.predict_proba(sample)


(array([1]), array([[0., 1.]]))

Backend will be receiving something like: 
```
{
  "window_start_min": 660,
  "window_end_min": 1080,
  "window_length_min": 420,
  "day": "tuesday",
  "weather": "windy",
  "occassion": "holiday"
}
```

In [67]:
# prediction helper 

def predict_smoke_risk(input_dict, threshold=0.25):
    """
    input_dict: dict with feature values
    returns: probability + binary alert
    """
    df_input = pd.DataFrame([input_dict])
    
    prob = pipeline.predict_proba(df_input)[0][1]
    alert = int(prob >= threshold)
    
    return {
        "smoke_probability": round(float(prob), 3),
        "alert": alert
    }

# API-ready

In [80]:
# example usage 

example_input = {
    "window_start_min": 1020,   # 5pm
    "window_end_min": 1200,     # 8pm
    "window_length_min": 180,
    "day": "friday",
    "weather": "windy",
    "occassion": "holiday"
}

predict_smoke_risk(example_input)

# output will look like this
# {
#   "smoke_probability": 0.23,
#   "alert": 0
# }



{'smoke_probability': 1.0, 'alert': 1}

In [1]:
import sklearn
print(sklearn.__version__)

1.6.1
